In [ ]:
# 1. Define
## Load lib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Define NN
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs):
        self.inputs = inputs  # Save inputs for backpropagation
        self.output = np.dot(inputs, self.weights) + self.biases  # Linear activation

    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)  # Gradient on weights
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)  # Gradient on biases
        self.dinputs = np.dot(dvalues, self.weights.T)  # Gradient on inputs


class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs  # Save inputs for backpropagation
        self.output = np.maximum(0, inputs)  # ReLU activation

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()  # Initialize gradients
        self.dinputs[self.inputs <= 0] = 0  # Zero gradient where input is negative


class Activation_SoftMax:
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        self.output = exp_values / np.sum(exp_values, axis=1, keepdims=True)

    def backward(self, dvalues):
        self.dinputs = dvalues  # For softmax + cross-entropy combined loss

class Loss:
    # Calculate the data and regulazation losses given
    # the output and the ground truth values
    def calculate(self,output,y):
        #Calculate sample losses:
        sample_losses = self.forward(output,y)
        #Calculate mean loss:
        data_loss = np.mean(sample_losses)
        #Return loss:
        return data_loss

class Loss_CategoricalCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)
        negative_log_likelihoods = -np.log(correct_confidences)
        return np.mean(negative_log_likelihoods)

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        labels = len(dvalues[0])
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]
        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples


class Optimizer_Adam:
    def __init__(self, learning_rate=0.001, decay=0., epsilon=1e-7, beta_1=0.9, beta_2=0.999):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.beta_1 = beta_1
        self.beta_2 = beta_2

    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    def update_params(self, layer):
        if not hasattr(layer, 'weight_cache'):
            layer.weight_momentums = np.zeros_like(layer.weights)
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_momentums = np.zeros_like(layer.biases)
            layer.bias_cache = np.zeros_like(layer.biases)

        layer.weight_momentums = self.beta_1 * layer.weight_momentums + (1 - self.beta_1) * layer.dweights
        layer.bias_momentums = self.beta_1 * layer.bias_momentums + (1 - self.beta_1) * layer.dbiases

        weight_momentums_corrected = layer.weight_momentums / (1 - self.beta_1 ** (self.iterations + 1))
        bias_momentums_corrected = layer.bias_momentums / (1 - self.beta_1 ** (self.iterations + 1))

        layer.weight_cache = self.beta_2 * layer.weight_cache + (1 - self.beta_2) * layer.dweights**2
        layer.bias_cache = self.beta_2 * layer.bias_cache + (1 - self.beta_2) * layer.dbiases**2

        weight_cache_corrected = layer.weight_cache / (1 - self.beta_2 ** (self.iterations + 1))
        bias_cache_corrected = layer.bias_cache / (1 - self.beta_2 ** (self.iterations + 1))

        layer.weights += -self.current_learning_rate * weight_momentums_corrected / (np.sqrt(weight_cache_corrected) + self.epsilon)
        layer.biases += -self.current_learning_rate * bias_momentums_corrected / (np.sqrt(bias_cache_corrected) + self.epsilon)

    def post_update_params(self):
        self.iterations += 1

class Activation_Softmax_Loss_CategoricalCrossEntropy():
    # Create activation and loss function object
    def __init__(self):
        self.activation = Activation_SoftMax()
        self.loss = Loss_CategoricalCrossentropy()

    # Forward pass
    def forward(self, inputs, y_true):
        # Output layer's activation function
        self.activation.forward(inputs)

        # Set the output
        self.output = self.activation.output

        # Calculate and return loss value
        return self.loss.calculate(self.output, y_true)
    
    # Backward pass
    def backward(self, dvalues, y_true):
        # number of samples
        samples = len(dvalues)

        #if labels are one-hot encoded, turn them into discrete values
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis =1)

        # Copy to safely modify
        self.dinputs = dvalues.copy()
        # Calculate gradients
        self.dinputs[range(samples), y_true] -= 1
        # Normalize gradients
        self.dinputs = self.dinputs / samples


# 3. Load data
mnist_train = pd.read_csv('mnist_train.csv', header=None).to_numpy()
mnist_test = pd.read_csv('mnist_test.csv', header=None).to_numpy()

X_train, y_train = mnist_train[:, 1:], mnist_train[:, 0]
X_test, y_test = mnist_test[:, 1:], mnist_test[:, 0]

# Normalize data
X_train = X_train / 255.0
X_test = X_test / 255.0

# 4. Train Neural network
input_dim = X_train.shape[1]

dense1 = Layer_Dense(input_dim, 128)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(128, 10)
softmax_cross_entropy = Activation_Softmax_Loss_CategoricalCrossEntropy()
optimizer = Optimizer_Adam(learning_rate=0.01)

epochs = 5
for epoch in range(epochs):
    dense1.forward(X_train)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    loss = softmax_cross_entropy.forward(dense2.output, y_train)

    predictions = np.argmax(softmax_cross_entropy.output, axis=1)
    accuracy = np.mean(predictions == y_train)
    
    print(f"Epoch {epoch + 1}, Loss: {loss:.3f}, Accuracy: {accuracy:.3f}")

    softmax_cross_entropy.backward(dense2.output, y_train)
    dense2.backward(softmax_cross_entropy.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()


In [ ]:
import matplotlib.pyplot as plt

def visualize(images, labels, predictions=None, num_samples=10):
    # Select random indices for visualization
    indices = np.random.choice(len(images), num_samples, replace=False)
    images = images[indices]
    labels = labels[indices]
    predictions = predictions[indices] if predictions is not None else None

    # Define the grid size
    grid_size = int(np.ceil(np.sqrt(num_samples)))

    # Create the figure
    plt.figure(figsize=(10, 10))
    for i in range(num_samples):
        plt.subplot(grid_size, grid_size, i + 1)
        plt.imshow(images[i].reshape(28, 28), cmap="gray")
        title = f"True: {labels[i]}"
        if predictions is not None:
            title += f"\nPred: {predictions[i]}"
        plt.title(title)
        plt.axis("off")
    plt.tight_layout()
    plt.show()


In [ ]:
visualize(X_train, y_train, num_samples=16)

In [ ]:
dense1.forward(X_test)
activation1.forward(dense1.output)
dense2.forward(activation1.output)

y_pred = np.argmax(dense2.output, axis=1)
visualize(X_test, y_test, predictions=y_pred, num_samples=16)
